# setup

In [ ]:
#from elasticsearch_dsl import Search, Q, Range, connections
#from elasticsearch.client import Elasticsearch
from tqdm.notebook import tqdm
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import os
import json
import re
from openai import OpenAI
import numpy as np

In [ ]:
EVENTS_FOLDER = "../data/events/"

In [ ]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 20)
pd.set_option('display.max_colwidth', None)

# browse_ai

In [ ]:
df_browse_ai = pd.read_pickle(f"{EVENTS_FOLDER}/browse_ai/df_browse_ai final 2025-01-07_18-08-29.pkl")

In [ ]:
df_browse_ai.rename(columns={"Event_category":"EventCategory"}, inplace=True)

In [ ]:
df_browse_ai.rename(columns={col: col.replace("extr", "") for col in df_browse_ai.columns}, inplace=True)

In [ ]:
df_browse_ai.columns

In [ ]:
df_browse_ai["lat"] = None
df_browse_ai["lon"] = None
df_browse_ai["Street"] = None
df_browse_ai["HouseNumber"] = None
df_browse_ai['StartTimeIsAssumed']=False

In [ ]:
df_browse_ai.drop(columns="MultipleDaysDetails", inplace=True)

In [ ]:
df_browse_ai.columns

# Gema

In [ ]:
df_gema = pd.read_pickle(f"{EVENTS_FOLDER}/gema/df_gema 2024-12-19_12-44-50.pkl")

In [ ]:
flt = df_gema.statusDescription=="Storniert"
df_gema = df_gema[~flt]

In [ ]:
df_gema.haeufigkeit.value_counts()

In [ ]:
df_gema['Source']="Gema"
df_gema['StartTimeIsAssumed']=True

for col in ["LocationDetails", "StartTime", "EndTime"]:
    df_gema[col] = None


In [ ]:
df_gema.rename(columns={"gemaNo_address_coordinate_lat":"lat", 
                        "gemaNo_address_coordinate_lon": "lon",
                        "gemaNo_address_streetHouseNumber": "StreetHouseNumber",
                        "gemaNo_address_city": "City",
                        "gemaNfNr": "origin_id",
                        "startDate": "StartDate",
                        "gemaNo_address_street": "Street",
                        "gemaNo_address_houseNumber": "HouseNumber",
                        "gemaNo_categoryType": "EventCategory",
                        "gemaNo_name": "LocationName",
                        "endDate":"EndDate"
                        }, inplace=True)

In [ ]:
pd.to_datetime(df_gema['EndDate'].dropna())

In [ ]:
flt = df_gema.EndDate == df_gema.StartDate
df_gema.loc[flt, "EndDate"] = None

In [ ]:
df_gema.StartDate = pd.to_datetime(df_gema.StartDate)
df_gema.EndDate = pd.to_datetime(df_gema.EndDate)

In [ ]:
df_gema["duration"] = (df_gema.EndDate - df_gema.StartDate).dt.days

In [ ]:
# df_gema.statusDescription.value_counts()

In [ ]:
# pd.set_option("display.max_rows", None)
# display(df_gema.loc[df_gema["duration"] > 60, "duration"].value_counts().sort_index())
# pd.set_option("display.max_rows", 20)


In [ ]:
#remove suspiciously long events
flt = (df_gema["duration"] <= 60) | (df_gema["duration"].isnull())
df_gema = df_gema[flt]

In [ ]:
len(df_gema)

In [ ]:
# pd.set_option("display.max_rows", None)
# display(df_gema.loc[:, "duration"].value_counts().sort_index())
# pd.set_option("display.max_rows", 20)


In [ ]:
# df_gema[df_gema.duration>=59]

In [ ]:
# df_gema.description.value_counts()

In [ ]:
# df_gema.EventCategory.value_counts().index

In [ ]:
#df_gema.statusDescription.value_counts()

In [ ]:
df_gema["isOnMultipleDays"]=False
df_gema.loc[df_gema["duration"]>0, "isOnMultipleDays"]=True

In [ ]:
df_gema.loc[df_gema.isOnMultipleDays==True, [ "StartDate", "EndDate", "duration"]]

In [ ]:
for col in df_browse_ai.columns:
    if col not in df_gema.columns:
        print(col)

In [ ]:
# for col in df_gema.columns:
#     if  "name" in col.lower():
#         print(col)

In [ ]:
# df_gema.gemaNo_name.value_counts()

In [ ]:
# df_gema.gemaNo_branche.value_counts()

In [ ]:
# df_gema.gemaNo_branchensegment.value_counts()

In [ ]:
# df_gema.gemaNo_categoryType.value_counts()

In [ ]:
# df_browse_ai.EventCategory.value_counts()

In [ ]:
# assume multiple events repeat every day. 

In [ ]:
# flt = ~df_gema.EndDate.isna()
# df_gema.loc[flt, "isOnMultipleDays"].value_counts()

In [ ]:
# flt = (~df_gema.EndDate.isna()) & (df_gema.isOnMultipleDays==False) 
# df_gema.loc[flt, "EndDate"].value_counts()

In [ ]:
# flt = (~df_gema.endDate.isna()) & (df_gema.isOnMultipleDays==False) # & (df_gema.StartDate!=df_gema.endDate)
# len(df_gema.loc[flt])

In [ ]:
# df_gema.duration.value_counts()

In [ ]:
# df_gema.lat.isna().value_counts()

In [ ]:
# df_gema.endDate.dropna()

In [ ]:
df_gema.columns

In [ ]:
cols_to_keep = df_browse_ai.columns.tolist()
df_gema = df_gema[cols_to_keep]

In [ ]:
df_gema_to_add = df_gema.loc[df_gema.isOnMultipleDays==False].copy()

In [ ]:
df_tmp = df_gema.loc[df_gema.isOnMultipleDays==True].copy()

In [ ]:
len(df_tmp)

In [ ]:
df_tmp

In [ ]:
rows_to_append = []
for i, row in df_gema.loc[df_gema.isOnMultipleDays==True].iterrows():
    start_date = row["StartDate"]
    end_date = row["EndDate"]
    print(f"{start_date=}")
    print(f"{end_date=}")
    date_range = pd.date_range(start_date, end_date)
    
    for date in date_range: 
        row_to_append = row.copy()
        row_to_append["StartDate"] = date.strftime('%Y-%m-%d')
        rows_to_append.append(row_to_append)

In [ ]:
pd.DataFrame(rows_to_append)

In [ ]:
df_gema_final = pd.concat([df_gema_to_add, pd.DataFrame(rows_to_append)], axis=0)

In [ ]:
df_gema_final

In [ ]:
df_gema_final.to_pickle(f"{EVENTS_FOLDER}/gema/df_gema after expanding multiple days events {datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.pkl")

In [ ]:
df_gema_final.columns

# apify

In [ ]:
# read a json file
with open(f"{EVENTS_FOLDER}apify/apify original DD FB.json") as f:
    apify = json.load(f)

In [ ]:
rows_to_append = []
for element in apify:
    event = element['event']
    location = element['location']
    location_address = location['address']
    row_to_append = {}
    # row_to_append["StartDate"] = event['startDateTime']
    # row_to_append["StartTime"] = event['startDateTime']
    # row_to_append["EndTime"] = event['endDateTime']
    # row_to_append["EndDate"] = event['endDateTime']
    row_to_append["StartDateTime"] = event['startDateTime']
    row_to_append["EndDateTime"] = event['endDateTime']
    row_to_append["City"] = location_address['city']
    row_to_append["LocationDetails"] = location_address['rawAddress']
    row_to_append["LocationName"] = location['name']
    row_to_append["Street"] = location_address['street']
    row_to_append["HouseNumber"] = location_address['houseNumber']
    row_to_append['origin_id'] = event['sourceInformation']['uuid']
    row_to_append['EventCategory'] = location['category'] if event['category'] is None else event['category']
    row_to_append['Source'] = event['sourceInformation']['platform']
    row_to_append['EventDescription'] = event['description']
    rows_to_append.append(row_to_append)

In [ ]:
df_apify = pd.DataFrame(rows_to_append)

In [ ]:
flt = df_apify.StartDateTime == df_apify.EndDateTime
df_apify.loc[flt, "EndDateTime"] = None


In [ ]:
for col in ['StreetHouseNumber', 'isOnMultipleDays', 'lat', 'lon', 'StartTimeIsAssumed', 'StartDate', 'EndDate', 'StartTime', 'EndTime']:
    df_apify[col] = None

In [ ]:
for col in df_gema_final.columns:
    if col not in df_apify.columns:
        print(col)

In [ ]:
df_apify.tail()

In [ ]:
df_apify.Source = df_apify.Source.str.replace("https://www.", "",  regex=False)

In [ ]:
# df_apify.EndDate.isna().value_counts()

In [ ]:
# df_apify.StartDate.isna().value_counts()

In [ ]:
# df_apify.StartDate.unique()

In [ ]:
for col in ['StartDateTime', 'EndDateTime']:
    df_apify[col] = df_apify[col].astype(str).str.replace("Z", "").str.replace("+02:00", "", regex=False).str.replace("+01:00", "", regex=False)
    #df_apify[col] = pd.to_datetime(df_apify[col], errors="coerce")

In [ ]:
df_apify.StartDateTime = pd.to_datetime(df_apify.StartDateTime, yearfirst=True)


In [ ]:
df_apify.EndDateTime = pd.to_datetime(df_apify.EndDateTime, yearfirst=True, errors="coerce")


In [ ]:
df_apify.StartDateTime.unique()

In [ ]:
df_apify.EndDateTime.unique()

In [ ]:
# flt = df_apify.EndDate.dt.year==2025
flt = df_apify.origin_id=="33a3e0d0-8141-11ef-aa2c-ed8353722086"
df_apify = df_apify[~flt]

In [ ]:
# for col in ['StartDate', "EndDate"]: 
#     df_apify[col] = df_apify[col].dt.strftime('%Y-%m-%d')


In [ ]:
# flt = df_apify.EndDate.isna()
# (df_apify.loc[~flt].EndTime-df_apify.loc[~flt].StartTime).sort_values()

In [ ]:
df_apify['isOnMultipleDays']=False

In [ ]:
# df_apify.StartTime = df_apify.StartTime.apply(lambda x: x.strftime('%H:%M'))

In [ ]:
# df_apify.EndTime = df_apify.EndTime.apply(lambda x: x.strftime('%H:%M') if x is not pd.NaT else None)

In [ ]:
# df_apify.EndTime.unique()

In [ ]:
# df_apify.StartTime.unique()

In [ ]:
df_apify.head()

In [ ]:
# flt =(~(df_apify.StartDate.str.contains("\+02:00"))) & (~(df_apify.StartDate.str.contains("\+01:00")))
# df_apify.loc[flt, ['StartDate', 'EventDescription', 'Source']]

In [ ]:
# apify[-1]

In [ ]:
# apify[0]['location']

In [ ]:
# apify[-1]['event']['description']

In [ ]:
df_events = pd.concat([df_browse_ai, df_gema_final, df_apify], axis=0)

In [ ]:
df_events.head()

In [ ]:
df_events.to_pickle(f"{EVENTS_FOLDER}/df_events have to add meslis {datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.pkl")